Train model

In [ ]:
# Provides similar functionality to ImageDataGenerators for videos
!pip install keras-video-generators

In [ ]:
# Please email me at nini16@tamu.edu if you do not ave access to the google drive.
# Permissions should have been granted but if not please email me!

from google.colab import drive
drive.mount('/content/drive/')
# drive.flush_and_unmount()

In [ ]:
# !rm -rf "/content/drive/MyDrive/CSCE636/Weights"

In [ ]:
# training data.
# Please ensure the file is present before running.
!unzip -q "/content/drive/MyDrive/CSCE636/new_train_2_classes.zip"

In [ ]:
import keras
from keras.regularizers import l2
from keras.preprocessing.image import load_img
import matplotlib.pyplot as plt
import numpy as np
from keras.layers import Conv2D, BatchNormalization, \
    MaxPool2D, GlobalMaxPool2D, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras_video import VideoFrameGenerator

from keras.layers import TimeDistributed, GRU, Dense, Dropout

from keras.models import load_model

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

import os
import numpy as np

import math

In [ ]:
# All frames should be resized
# Please select a batch-size that divides the number of samples!
# THE DATASET WILL PROBABLY BE INCREASED FOR THE NEXT SUBMISSION SO
# BE SURE TO ADJUST THE BATCH SIZE!!

img_shape = (224, 224)
BS = 32

In [ ]:
# Apply image augmentation to each frame
# Please confirm that this directory is present before running

vid_gen = VideoFrameGenerator(
    glob_pattern=r"/content/new_train_2_classes/{classname}/*",
    nb_frames=5,
    split_val=.15, 
    shuffle=True,
    batch_size=BS,
    target_shape=img_shape,
    nb_channel=3,
    transformation=ImageDataGenerator(rescale=1./255,
                                      rotation_range=30,
                                      # width_shift_range=0.1,
                                      # height_shift_range=0.1,
                                      # shear_range=0.1,
                                      zoom_range=0.1,
                                      horizontal_flip=True,
                                      fill_mode="nearest"),
    use_frame_cache=False)

In [ ]:
validation_gen = vid_gen.get_validation_generator()

In [ ]:
# Can use this for visualization
# from keras_video import utils
# utils.show_sample(vid_gen, random=True)

In [ ]:
# model structure for Feature Extractor
def build_convnet_2(shape=(224, 224, 3)):
    momentum = .9
    model = keras.Sequential()
    model.add(Conv2D(64, (3,3), input_shape=shape,
        padding='same', activation='relu', name='conv1'))
    model.add(Conv2D(64, (3,3), padding='same', activation='relu', name='conv2'))
    model.add(BatchNormalization(momentum=momentum))
    
    model.add(MaxPool2D())
    
    model.add(Conv2D(128, (3,3), padding='same', activation='relu', name='conv3'))
    model.add(Conv2D(128, (3,3), padding='same', activation='relu', name='conv4'))
    model.add(BatchNormalization(momentum=momentum))
    
    model.add(MaxPool2D())
    
    model.add(Conv2D(256, (3,3), padding='same', activation='relu', name='conv5'))
    model.add(Conv2D(256, (3,3), padding='same', activation='relu', name='conv6'))
    model.add(Conv2D(256, (3,3), padding='same', activation='relu', name='conv7'))
    model.add(BatchNormalization(momentum=momentum))
    
    model.add(MaxPool2D())
    
    model.add(Conv2D(512, (3,3), padding='same', activation='relu', name='conv8'))
    model.add(Conv2D(512, (3,3), padding='same', activation='relu', name='conv9'))
    model.add(Conv2D(512, (3,3), padding='same', activation='relu', name='conv10'))
    model.add(BatchNormalization(momentum=momentum))

    model.add(MaxPool2D())

    model.add(Conv2D(512, (3,3), padding='same', activation='relu', name='conv11'))
    model.add(Conv2D(512, (3,3), padding='same', activation='relu', name='conv12'))
    model.add(Conv2D(512, (3,3), padding='same', activation='relu', name='conv13'))
    model.add(BatchNormalization(momentum=momentum))
    
    # flatten...
    model.add(GlobalMaxPool2D())

    model.add(Dense(512, activation='relu', name='fc1', kernel_regularizer=l2(0.01)))
    model.add(Dropout(.5))
    model.add(Dense(256, activation='relu', name='fc2', kernel_regularizer=l2(0.01)))
    model.add(Dropout(.5))
    model.add(Dense(8, activation='softmax', name='fc3'))

    model.load_weights("/content/drive/MyDrive/CSCE636/Feature_extract_weights_194-1.12.hdf5")
    for layer in model.layers:
        layer.trainable = False

    # Removes fully-connected layers
    for i in range(4):
      model.pop()

    return model

In [ ]:
def action_model(shape=(5,) + img_shape + (3,), nbout=3):
    # Create our feature extractor convnet with img_shape input shape
    convnet = build_convnet_2()
    
    # then create our final model
    model = keras.Sequential()
    # add the convnet with img_shape shape
    model.add(TimeDistributed(convnet, input_shape=shape))
    # add GRU
    model.add(GRU(64))
    # and finally, we make a decision network
    model.add(Dense(1024, activation='relu', kernel_regularizer=keras.regularizers.l2(l2=0.01)))
    model.add(Dropout(.2))
    model.add(Dense(2, activation='softmax'))

    model.summary()
    return model

In [ ]:
# instantiate and compile model
model = action_model()
optimizer = keras.optimizers.Adam(0.0001)
model.compile(
    optimizer,
    'categorical_crossentropy',
    metrics=['acc']
)

In [ ]:
# Adjust epochs and other parameters as needed
# Callbacks have been commented out to avoid overwriting existin data.
# Whoever is running this can uncomment them as needed

callbacks = [
    # keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=7, min_lr=0.000001),
    # keras.callbacks.EarlyStopping(
    #     monitor='val_acc',
    #     patience=20,
    #     ),
    # keras.callbacks.ModelCheckpoint(
    #     '/content/drive/MyDrive/CSCE636/Weights/weights.{epoch:02d}-{val_loss:.2f}.hdf5',
    #     monitor='val_acc',
    #     save_best_only=True,
    #     verbose=1),
]

history = model.fit_generator(
    vid_gen,
    steps_per_epoch=math.ceil(526/BS),
    validation_data=validation_gen,
    verbose=1,
    epochs=1, # last used 80,
    shuffle=True,
    callbacks=callbacks
)

In [ ]:
# uncomment only if you need to
# model.save('/content/drive/MyDrive/CSCE636/main_model.h5')

In [ ]:
# uncomment only if you need to
# np.save('/content/drive/MyDrive/CSCE636/train_history_main_model.npy',history.history)



Testing model

In [ ]:
# just in case
!pip install tqdm

In [ ]:
def ExtractFrames(file_path, pos=[0.1,0.3,0.5,0.7,0.9]):
    # Extracts frames from file_path at the positions (relative between 0 and 1) in pos
    
    import os
    
    if not len(pos):
        print("[ExtractFrames]: Invalid positions")
        return None
    
    if not os.path.isfile(file_path) :
        print("[ExtractFrames]: Invalid file path")
        return None
    
    import cv2
    
    # container for frames
    arr = np.empty((len(pos),224,224,3))
    
    cap = cv2.VideoCapture(file_path)
    total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    
    for k,i in enumerate(pos):
        # get frame number
        position = int(i * total_frames)
        
        # set frame pointer at i and extract frame
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        
        # preprocessing
        frame = cv2.resize(frame, (224,224))
        frame = frame * 1/255.
        frame = np.float32(frame)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # insert in container
        arr[k] = frame
        
    # cleanup
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    cap.release()
    
    return arr

In [ ]:
def test_flow_from_directory(dir_path):
    import numpy as np
    import os
    from tqdm import tqdm
    
    # 2 classes: "brushing_teeth", "miscellaneous"
    posDIR = os.path.join(dir_path, "brushing_teeth")
    negDIR = os.path.join(dir_path, "miscellaneous")
    
    pos_samples = os.listdir(posDIR)
    neg_samples = os.listdir(negDIR)
    
    # Get all the videos in both classes
    pos_samples = [os.path.join(posDIR, fname) for fname in pos_samples]
    neg_samples = [os.path.join(negDIR, fname) for fname in neg_samples]
    
    sample_size = len(pos_samples) + len(neg_samples)
    
    # Array for data, labels and files 
    test_data   = np.empty( (sample_size, 5, 224, 224, 3) )
    test_label  = np.empty( (sample_size, 1) )
    test_files  = pos_samples+neg_samples
    
    # Extract frames from all videos using default positions
    index = 0
    print("Now extracting brushing_teeth videos")
    for vid in tqdm(pos_samples):
        test_data[index] = ExtractFrames(str(vid))
        test_label[index] = 1.
        index += 1
    
    print("Now extracting miscellaneous videos")
    for vid in tqdm(neg_samples):
        test_data[index] = ExtractFrames(vid)
        test_label[index] = 0.
        index += 1
    
    return test_data, test_label, test_files

In [ ]:
!unzip -q "/content/drive/MyDrive/CSCE636/YoutubeTest_v2.zip"

In [ ]:
model = load_model("/content/drive/MyDrive/CSCE636/main_model.h5")

In [ ]:
# Loads data, label and filenames
test_data, test_label, test_files = test_flow_from_directory("/content/YoutubeTest_v2")
test_label = np.reshape(test_label, test_label.shape[0])

In [ ]:
BS = 50
num_steps = math.ceil(test_data.shape[0]/BS)
num_steps

In [ ]:
pred = model.predict(test_data,verbose=1,batch_size=50, steps=num_steps)

In [ ]:
index_max = np.argmax(pred, axis=1)
# "brushing_teeth" - 1, "not brushing_teeth" - 0
# if argmax is index 0, then it predicted brushing teeth, hence
# assign a 1 or else assign a 0
lookup = {1:0, 0:1}
predicted_labels = np.array([lookup[i] for i in index_max])

In [ ]:
accuracy_score(test_label, predicted_labels)

In [ ]:
confusion_matrix(test_label, predicted_labels)

In [ ]:
print("False Negative Rate: {}".format(75/(319+75)))

In [ ]:
print("False Positive Rate: {}".format(145/(214+145)))

In [ ]:
# import csv
# fields = ['file', 'Label']
# expData = []
# for i in range(394):
#     expData.append([test_files[i], predicted_labels[i]])

In [ ]:
# with open('/content/filecheck.csv', 'w') as f: 
      
#     # using csv.writer method from CSV package 
#     write = csv.writer(f) 
      
#     write.writerow(fields) 
#     write.writerows(expData)